In [1]:
import os
import nltk
import re
import numpy as np
import yaml
import glob
import matplotlib.pyplot as plt

In [2]:
data_path = '/home/beiming/RAW_DATA/MEG/case0554_A2_220715/Sentences/Baseline'
phone_list_path = '/home/beiming/RAW_DATA/MEG/case0554_A2_220715/Sentences/PB400_phone.txt'
config = yaml.load(open('conf/conf.yaml', 'r'), Loader=yaml.FullLoader)

In [3]:
meg_list = glob.glob(data_path + '/*.txt')
meg_list.sort()

sample_meg = np.loadtxt(meg_list[0])

dim_range = config['MEG_data']['selected_dims']

In [ ]:
class MEG_dim_selection(object):
    def __init__(self, dim_range):
        self.dim_range = dim_range
    def __call__(self, X):
        if len(dim_range) % 2 != 0:
            raise Exception("Nums of starts and ends don't match, please check!")
        sel = []   
        for i in range(len(dim_range)//2):
            start = dim_range[2*i]
            end = dim_range[2*i+1]            
            _X = X[:,start:end]           
            sel.append(_X)   
        X_sel = np.hstack(sel)
        return X_sel
    
dim_sel = MEG_dim_selection(dim_range)
X_sel = dim_sel(sample_meg)

print(X_sel.shape)

In [ ]:
class low_pass_filtering(object):
    def __init__(self, cutoff_freq, fs):
        
        self.fs = fs
        self.cutoff_freq = cutoff_freq
    def __call__(self, I):
        if self.cutoff_freq >= self.fs//2:
            raise Exception("Cutoff freq must be lower than half of the sampling rate, based on the Nyquist criterion")
        from scipy.signal import butter, lfilter
        cutoff_norm = self.cutoff_freq/(self.fs/2)
        b, a = butter(5, cutoff_norm, btype='low', analog=False)
        I_filtered_list = []
        for i in range(I.shape[1]):
            _I = lfilter(b, a, I[:,i])
            I_filtered_list.append(_I)
            I_filtered = np.vstack(I_filtered_list)

        return I_filtered.T
    
LP = low_pass_filtering(200, 1000)
S_F = LP(sample_meg)

In [ ]:
fs = 36
plt.figure(figsize=(18,6), dpi=300)
x1 = sample_meg[:,0]
plt.plot(x1, 'r') 

In [ ]:
fs = 36
plt.figure(figsize=(18,6), dpi=300)
x1 = S_F[:,0]
plt.plot(x1, 'r') 

In [39]:
# (3835, 204)

class MEG_framing(object):
    def __init__(self, frame_len, frame_shift, drop_last = True):
        
        self.frame_len = frame_len
        self.frame_shift = frame_shift
        self.drop_last = drop_last
    def __call__(self, I):
        frame_overlap = self.frame_len - self.frame_shift
        
        num_frames = np.abs(I.shape[0] - frame_overlap) // np.abs(self.frame_len - frame_overlap)
        rest_samples = np.abs(I.shape[0] - frame_overlap) % np.abs(self.frame_len - frame_overlap)
                
        if rest_samples != 0:
            pad_signal_length = int(self.frame_shift - rest_samples)
            z = np.zeros((pad_signal_length,I.shape[1]))
            pad_signal = np.append(I, z, axis = 0)
            num_frames += 1
        else:
            pad_signal = I

        idx1 = np.tile(np.arange(0, self.frame_len), (num_frames, 1))
        idx2 = np.tile(np.arange(0, num_frames * self.frame_shift, self.frame_shift),(self.frame_len, 1)).T
        indices = idx1 + idx2
        frames = pad_signal[indices.astype(np.int32, copy=False),:]
        
        if self.drop_last == True:
            frames = frames[:-1,:,:]
        
        return frames

MEG_Fr = MEG_framing(11,10)
S_F = MEG_Fr(sample_meg)
print(S_F.shape)

(383, 11, 204)


In [ ]:
dim_sel = MEG_dim_selection(dim_range)
LP = low_pass_filtering(200, 1000)
MEG_Fr = MEG_framing(20, 5)

In [ ]:
transforms = []
transforms.append(dim_sel)
transforms.append(LP)
transforms.append(MEG_Fr)

class Transform_Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, X):
        for t in self.transforms:
            X = t(X)
        return X
    
transforms_all = Transform_Compose(transforms)

In [41]:
CV_num = 8
CV_list = list(range(CV_num))
print(CV_list)

[0, 1, 2, 3, 4, 5, 6, 7]


In [ ]:
train_spk_list = spk_list.copy()
train_spk_list.remove(spk_list[i])

In [42]:
label_path = '/home/beiming/RAW_DATA/MEG/case0554_A2_220715/Sentences/PB400_phone.txt'

f = open(label_path, 'r')
ls = f.readlines()
print(ls)


['DH AH W AA CH D AO G G EY V EY W AO R N IH NG G R AW L\n', 'SH IY M EY D DH AH B EH D W IH DH K L IY N SH IY T S\n', 'DH AH OW L D M AE N D IH S K AH S T DH AH D AY V\n', 'B AA B HH ER D P AO L K AO L D AH B AW T DH AH S T R IH P S\n', 'AY SH UH D HH AE V K AH N S IH D ER D DH AH M AE P\n', 'DH AH OW L D T R EY N W AA Z P AW ER D B AY S T IY M\n', 'HH IY K AA T DH AH F IH SH IH N HH IH Z N EH T\n', 'M IH S B R AW N SH UH D AH N T D IH S K AH S DH AH S AE N D\n', 'K L OW S DH AH W IH N D OW T UW S T AA P DH AH D R AE F T\n', 'M AY T IY V IY HH AE Z EY T W EH L V IH N CH S K R IY N\n', 'DH EY M AY T HH AE V K AH N S IH D ER D DH AH HH AY V\n', 'D EY V IH D HH AE Z D IH S K AH S T DH AH D EH N T\n', 'DH AH S AE N D AH L HH AE Z EY B R OW K AH N S T R AE P\n', 'DH AH B OW T S EY L D AH L AO NG DH AH K OW S T\n', 'K R AA K AH D AY L Z L AY V IH N M AH D IY S W AA M P S\n', 'HH IY K AE N T K AH N S IH D ER DH AH K R IH B\n', 'DH AH F AA R M ER HH AA R V AH S T IH D HH IH Z K R AA P\n', 'AO